CNN model traind using coirdinates data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Load the dataset
file_path = r"dataset.csv"
df = pd.read_csv(file_path)

# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

# Separate features and labels
labels = df['class']
features = df.drop(columns=['class'])

# Ensure all features are numeric
features = features.apply(pd.to_numeric, errors='coerce')
features = features.fillna(0)

# Convert the DataFrame to a NumPy array
features_array = features.to_numpy(dtype=np.float32)

# **Flatten the features array** to ensure compatibility with the model's expected input shape
# Each row should be a vector of length 99 if you have 33 keypoints with x, y, z coordinates
features_array = features_array.reshape(features_array.shape[0], -1)  # Shape becomes (samples, 99)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# One-hot encode the labels
labels_categorical = to_categorical(labels_encoded)

# Split the dataset into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(features_array, labels_categorical, test_size=0.3, random_state=42)

# Define a simple classifier model on top of the flattened features
def create_classifier_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        Input(shape=(input_shape,)),  # Ensure the input shape is (99,)
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# K-Fold Cross-Validation
k = 4  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

val_accuracies = []
val_losses = []
train_accuracies = []
train_losses = []
test_accuracies = []
test_losses = []

for fold, (train_index, val_index) in enumerate(kf.split(X_train_full)):
    print(f"Fold {fold + 1}")
    
    X_train, X_val = X_train_full[train_index], X_train_full[val_index]
    y_train, y_val = y_train_full[train_index], y_train_full[val_index]
    
    # Create and train the classifier model
    model = create_classifier_model(X_train.shape[1], labels_categorical.shape[1])
    history = model.fit(X_train, y_train, epochs=30, batch_size=5, validation_data=(X_val, y_val), callbacks=[early_stopping], shuffle=True)
    
    # Evaluate the model on training set
    train_loss, train_acc = model.evaluate(X_train, y_train)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Evaluate the model on validation set
    val_loss, val_acc = model.evaluate(X_val, y_val)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # Evaluate the model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    # Print metrics for the current fold
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# Calculate mean and standard deviation for training, validation, and test metrics
mean_train_loss = np.mean(train_losses)
std_train_loss = np.std(train_losses)
mean_train_acc = np.mean(train_accuracies)
std_train_acc = np.std(train_accuracies)

mean_val_loss = np.mean(val_losses)
std_val_loss = np.std(val_losses)
mean_val_acc = np.mean(val_accuracies)
std_val_acc = np.std(val_accuracies)

mean_test_loss = np.mean(test_losses)
std_test_loss = np.std(test_losses)
mean_test_acc = np.mean(test_accuracies)
std_test_acc = np.std(test_accuracies)

# Print final metrics
print("\nFinal Metrics:")
print(f"Training Loss: {mean_train_loss:.4f} ± {std_train_loss:.4f}")
print(f"Training Accuracy: {mean_train_acc:.4f} ± {std_train_acc:.4f}")
print(f"Validation Loss: {mean_val_loss:.4f} ± {std_val_loss:.4f}")
print(f"Validation Accuracy: {mean_val_acc:.4f} ± {std_val_acc:.4f}")
print(f"Test Loss: {mean_test_loss:.4f} ± {std_test_loss:.4f}")
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_test_acc:.4f}")

# Save the last trained model
model.save("cnn_model_new3.keras")

# Plot training and validation accuracy for the last fold
plt.figure(figsize=(10, 2))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Plot training and validation loss for the last fold
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# Check class distribution in the test set
test_class_distribution = np.argmax(y_test, axis=1)
print("Test set class distribution:", np.bincount(test_class_distribution))

# Check class distribution in the predictions
predictions = np.argmax(model.predict(X_test), axis=1)
print("Predicted class distribution:", np.bincount(predictions))

# Generate confusion matrix
conf_matrix = confusion_matrix(test_class_distribution, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show()


CNN model train ed using image data

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.model_selection import KFold


# Paths to the image dataset directories
train_dir = "Dataset/train"
val_dir = "Dataset/test"

# Define image size and batch size
img_height, img_width = 224,224
batch_size = 16

# Set up ImageDataGenerator with data augmentation for train and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # Ensure data is shuffled
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No shuffle for validation data to maintain consistency
)

# Set up the modified CNN model with dropout and batch normalization
def create_cnn_model(input_shape, num_classes):
    with tf.device('/CPU:0'):
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
            BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
    
            
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
            
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
            

            tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
           
            
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            BatchNormalization(),
            
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

# Initialize the model
num_classes = train_data.num_classes
model = create_cnn_model((img_height, img_width, 3), num_classes)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-3 * 0.9 ** epoch)
# Train the model with the generator
history = model.fit(
    train_data,
    epochs=25,  # Increased epochs to allow more training with early stopping
    validation_data=val_data,
    callbacks=[early_stopping, lr_scheduler]
)

# Plot training and validation accuracy
plt.figure(figsize=(10, 2))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Predictions and Confusion Matrix
val_data.reset()  # Reset generator for reproducible results
predictions = np.argmax(model.predict(val_data), axis=-1)
true_labels = val_data.classes

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=val_data.class_indices.keys(), yticklabels=val_data.class_indices.keys())
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show()

# Save the trained model
model.save("cnn_model_augmented_dropout2.keras")
